In [2]:
import pandas as pd
import os

In [3]:
FILE_PATH = os.path.join(os.path.abspath(''), '..', 'raw_data', 'population_data_2010-2019.csv')
raw_teens = pd.read_csv(FILE_PATH, engine='python')

FILE_PATH = os.path.join(os.path.abspath(''), '..', 'raw_data', 'population_data_2000-2010.csv')
raw_aughts = pd.read_csv(FILE_PATH, engine='python')


In [4]:
teens = raw_teens[['NAME', 'STNAME', 'POPESTIMATE2010', 'POPESTIMATE2011',
       'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014',
       'POPESTIMATE2015', 'POPESTIMATE2016', 'POPESTIMATE2017',
       'POPESTIMATE2018', 'POPESTIMATE2019']]
teens.columns = ['name', 'state', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']
teens = teens.drop_duplicates()

In [5]:
aughts = raw_aughts[['NAME', 'STNAME', 'POPESTIMATE2000', 'POPESTIMATE2001',
       'POPESTIMATE2002', 'POPESTIMATE2003', 'POPESTIMATE2004',
       'POPESTIMATE2005', 'POPESTIMATE2006', 'POPESTIMATE2007',
       'POPESTIMATE2008', 'POPESTIMATE2009']]
aughts.columns = ['name', 'state', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009']
aughts = aughts.drop_duplicates()

In [6]:
merged = teens.merge(aughts, on=['name', 'state'], how='left')
merged

,name,state,2010,2011,2012,2013,2014,2015,2016,2017,...,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009
0,Alabama,Alabama,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,...,4452173.0,4467634.0,4480089.0,4503491.0,4530729.0,4569805.0,4628981.0,4672840.0,4718206.0,4757938.0
1,Abbeville city,Alabama,2699,2694,2643,2628,2608,2600,2584,2575,...,2985.0,2941.0,2909.0,2882.0,2857.0,2820.0,2807.0,2784.0,2742.0,2714.0
2,Adamsville city,Alabama,4500,4493,4471,4449,4420,4390,4356,4327,...,5021.0,4960.0,4894.0,4841.0,4784.0,4728.0,4687.0,4633.0,4594.0,4558.0
3,Addison town,Alabama,751,750,743,742,739,734,731,726,...,701.0,701.0,708.0,714.0,722.0,729.0,741.0,750.0,752.0,759.0
4,Akron town,Alabama,355,347,347,343,338,339,333,332,...,485.0,473.0,454.0,442.0,426.0,416.0,404.0,395.0,384.0,369.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99274,Balance of Uinta County,Wyoming,4813,4793,4827,4828,4807,4807,4812,4762,...,4561.0,4497.0,4531.0,4501.0,4493.0,4493.0,4537.0,4638.0,4734.0,4830.0
99275,Washakie County,Wyoming,8530,8449,8409,8413,8273,8278,8165,8010,...,8252.0,8068.0,7988.0,7976.0,7960.0,8022.0,7979.0,8169.0,8229.0,8423.0
99276,Balance of Washakie County,Wyoming,2782,2755,2734,2737,2690,2690,2650,2599,...,2539.0,2500.0,2490.0,2502.0,2514.0,2548.0,2550.0,2626.0,2660.0,2739.0
99277,Weston County,Wyoming,7198,7142,7077,7136,7138,7208,7220,6968,...,6627.0,6487.0,6578.0,6610.0,6646.0,6594.0,6717.0,7033.0,7133.0,7266.0


In [7]:
# creating a feature that will tell if entry is a city, state or county
merged['type'] = merged['name']

# removing all of the county balances that are not part of any city
merged['type'] = merged['type'].apply(lambda x: 'balance' if x.startswith('Balance of') else x)
merged = merged[merged['type'] != 'balance']

In [8]:
# seperating out the counties
merged['type'] = merged['type'].apply(lambda x: 'county' if x.endswith('County') else x)
counties = merged[merged['type'] == 'county']
counties = counties.drop('type', axis=1)
merged = merged[merged['type'] != 'county']
counties.head()

,name,state,2010,2011,2012,2013,2014,2015,2016,2017,...,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009
463,Autauga County,Alabama,54773,55227,54954,54727,54893,54864,55243,55390,...,44021.0,44889.0,45909.0,46800.0,48366.0,49676.0,51328.0,52405.0,53277.0,54135.0
469,Baldwin County,Alabama,183112,186558,190145,194885,199183,202939,207601,212521,...,141342.0,144875.0,147957.0,151509.0,156266.0,162183.0,168121.0,172404.0,175827.0,179406.0
471,Barbour County,Alabama,27327,27341,27169,26937,26755,26283,25806,25157,...,29015.0,28863.0,28653.0,28594.0,28287.0,28027.0,27861.0,27757.0,27808.0,27657.0
473,Bibb County,Alabama,22870,22745,22667,22521,22553,22566,22586,22550,...,19913.0,21028.0,21199.0,21399.0,21721.0,22042.0,22099.0,22438.0,22705.0,22941.0
479,Blount County,Alabama,57376,57560,57580,57619,57526,57526,57494,57787,...,51107.0,51845.0,52551.0,53457.0,54124.0,54624.0,55485.0,56240.0,57055.0,57341.0


In [9]:
county_population = counties[['name', 'state', '2000', '2001', '2002', '2003', '2004',
                                '2005', '2006', '2007', '2008', '2009', '2010', '2011',
                                '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']]

In [47]:
# creating the basic county data
county_names = counties[['name', 'state']]
county_names.columns = ['county_name', 'state_name']

county_id = raw_teens[['STATE', 'COUNTY', 'NAME', 'STNAME']]
county_id['STATE'] = county_id['STATE'].apply(lambda x: str(x).zfill(2))
county_id['COUNTY'] = county_id['COUNTY'].apply(lambda x: str(x).zfill(3))
county_id['county_id'] = county_id['STATE'] + county_id['COUNTY']
county_id = county_id[['STATE', 'NAME', 'STNAME', 'county_id']]
county_id.columns = ['state_id', 'county_name', 'state_name', 'county_id']
county_basic = county_names.merge(county_id, on=['county_name', 'state_name'], how='left')
county_basic = county_basic[['county_id', 'county_name', 'state_id']]
county_basic

,county_id,county_name,state_id
0,01001,Autauga County,01
1,01003,Baldwin County,01
2,01005,Barbour County,01
3,01007,Bibb County,01
4,01009,Blount County,01
...,...,...,...
3008,56037,Sweetwater County,56
3009,56039,Teton County,56
3010,56041,Uinta County,56
3011,56043,Washakie County,56


In [10]:
# seperating out the states
states_list = set(merged['state'].to_list())
merged['type'] = merged['type'].apply(lambda x: 'state' if x in states_list else x)
states = merged[merged['type'] == 'state']
states = states.drop('type', axis=1)
merged = merged[merged['type'] != 'state']
states.head()

,name,state,2010,2011,2012,2013,2014,2015,2016,2017,...,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009
0,Alabama,Alabama,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,...,4452173.0,4467634.0,4480089.0,4503491.0,4530729.0,4569805.0,4628981.0,4672840.0,4718206.0,4757938.0
787,Alaska,Alaska,713910,722128,730443,737068,736283,737498,741456,739700,...,627963.0,633714.0,642337.0,648414.0,659286.0,666946.0,675302.0,680300.0,687455.0,698895.0
987,Arizona,Arizona,6407172,6472643,6554978,6632764,6730413,6829676,6941072,7044008,...,5160586.0,5273477.0,5396255.0,5510364.0,5652404.0,5839077.0,6029141.0,6167681.0,6280362.0,6343154.0
1141,Arkansas,Arkansas,2921964,2940667,2952164,2959400,2967392,2978048,2989918,3001345,...,2678588.0,2691571.0,2705927.0,2724816.0,2749686.0,2781097.0,2821761.0,2848650.0,2874554.0,2896843.0
1874,California,California,37319502,37638369,37948800,38260787,38596972,38918045,39167117,39358497,...,33987977.0,34479458.0,34871843.0,35253159.0,35574576.0,35827943.0,36021202.0,36250311.0,36604337.0,36961229.0


In [23]:
state_population = states[['state', '2000', '2001', '2002', '2003', '2004',
                                '2005', '2006', '2007', '2008', '2009', '2010', '2011',
                                '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']]

In [95]:
# creating the basic state data
state_names = states[['state']]
state_names.columns = ['state_name']

state_id = raw_teens[['STATE', 'STNAME']]
state_id.columns = ['state_id', 'state_name']

state_basic = state_names.merge(state_id, on='state_name', how='left')
state_basic = state_basic.drop_duplicates(ignore_index=True)
state_basic['state_id'] = state_basic['state_id'].apply(lambda x: str(x).zfill(2))

state_abbreviations = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

state_basic['state_abbreviation'] = state_basic['state_name'].map(state_abbreviations)
state_basic

,state_name,state_id,state_abbreviation
0,Alabama,01,AL
1,Alaska,02,AK
2,Arizona,04,AZ
3,Arkansas,05,AR
4,California,06,CA
5,Colorado,08,CO
6,Connecticut,09,CT
7,Delaware,10,DE
8,District of Columbia,11,DC
9,Florida,12,FL


In [12]:
# seperating out the cities
types_that_are_cities = ['city', 'town', 'village']
merged['type'] = merged['name']
merged['type'] = merged['type'].apply(lambda x: 'city' if x.split()[-1] in types_that_are_cities else x)
cities = merged[merged['type'] == 'city']
cities = cities.drop('type', axis=1)
cities['name'] = cities['name'].apply(lambda x: x.replace(' city', ''))
cities['name'] = cities['name'].apply(lambda x: x.replace(' town', ''))
cities['name'] = cities['name'].apply(lambda x: x.replace(' village', ''))
cities.head()

,name,state,2010,2011,2012,2013,2014,2015,2016,2017,...,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009
1,Abbeville,Alabama,2699,2694,2643,2628,2608,2600,2584,2575,...,2985.0,2941.0,2909.0,2882.0,2857.0,2820.0,2807.0,2784.0,2742.0,2714.0
2,Adamsville,Alabama,4500,4493,4471,4449,4420,4390,4356,4327,...,5021.0,4960.0,4894.0,4841.0,4784.0,4728.0,4687.0,4633.0,4594.0,4558.0
3,Addison,Alabama,751,750,743,742,739,734,731,726,...,701.0,701.0,708.0,714.0,722.0,729.0,741.0,750.0,752.0,759.0
4,Akron,Alabama,355,347,347,343,338,339,333,332,...,485.0,473.0,454.0,442.0,426.0,416.0,404.0,395.0,384.0,369.0
5,Alabaster,Alabama,31209,31375,31684,31980,32182,32772,33017,33275,...,24116.0,24674.0,25223.0,25957.0,26739.0,27504.0,28462.0,29020.0,29630.0,30107.0


In [13]:
city_population = cities[['name', 'state', '2000', '2001', '2002', '2003', '2004',
                                '2005', '2006', '2007', '2008', '2009', '2010', '2011',
                                '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']]

In [109]:
# creating basic city data
city_names = cities[['name', 'state']]
city_names.columns = ['city_name', 'state_name']

city_id = raw_teens[['STATE', 'COUNTY', 'PLACE', 'NAME', 'STNAME']]
city_id['STATE'] = city_id['STATE'].apply(lambda x: str(x).zfill(2))
city_id['COUNTY'] = city_id['COUNTY'].apply(lambda x: str(x).zfill(3))
city_id['county_id'] = city_id['STATE'] + city_id['COUNTY']
city_id['NAME'] = city_id['NAME'].apply(lambda x: x.replace(' city', ''))
city_id['NAME'] = city_id['NAME'].apply(lambda x: x.replace(' town', ''))
city_id['NAME'] = city_id['NAME'].apply(lambda x: x.replace(' village', ''))
city_id.columns = ['state_id', 'COUNTY', 'PLACE', 'city_name', 'state_name', 'county_id']
city_basic = city_names.merge(city_id, on=['city_name', 'state_name'])
city_basic = city_basic.drop_duplicates()
city_basic = city_basic[city_basic['COUNTY'] != '000']
city_basic['PLACE'] = city_basic['PLACE'].apply(lambda x: str(x).zfill(5))
city_basic['city_id'] = city_basic['state_id'] + '-' + city_basic['PLACE']
city_basic = city_basic[['city_id', 'city_name', 'county_id', 'state_id']]
city_basic

,city_id,city_name,county_id,state_id
1,01-00124,Abbeville,01067,01
3,01-00460,Adamsville,01073,01
5,01-00484,Addison,01133,01
7,01-00676,Akron,01065,01
9,01-00820,Alabaster,01117,01
...,...,...,...,...
61483,56-81300,Wamsutter,56037,56
61485,56-83040,Wheatland,56031,56
61487,56-84925,Worland,56043,56
61489,56-85015,Wright,56005,56


In [104]:
# adding in latitude and longitude
FILE_PATH = os.path.join(os.getcwd(), '..', 'raw_data', 'basic_city_data.txt')
city_geographic = pd.read_csv(FILE_PATH, sep='\t')
city_geographic = city_geographic[['USPS', 'NAME', 'ALAND_SQMI', 'AWATER_SQMI', 'INTPTLAT', 'INTPTLONG                                                                                                 ']]
city_geographic.columns = ['state_abbreviation', 'city_name', 'land_sqmi', 'water_sqmi', 'lat', 'lon']
city_geographic = city_geographic.merge(state_basic, on='state_abbreviation')
city_geographic = city_geographic.drop(['state_abbreviation', 'state_name'], axis=1)
city_geographic['type'] = city_geographic['city_name']
types_that_are_cities = ['city', 'town', 'village']
city_geographic['type'] = city_geographic['type'].apply(lambda x: 'city' if x.split()[-1] in types_that_are_cities else x)
city_geographic = city_geographic[city_geographic['type'] == 'city']
city_geographic['city_name'] = city_geographic['city_name'].apply(lambda x: x.replace(' city', ''))
city_geographic['city_name'] = city_geographic['city_name'].apply(lambda x: x.replace(' town', ''))
city_geographic['city_name'] = city_geographic['city_name'].apply(lambda x: x.replace(' village', ''))
city_geographic = city_geographic.drop('type', axis=1)
city_geographic


,city_name,land_sqmi,water_sqmi,lat,lon,state_id
1,Abbeville,15.543,0.042,31.564724,-85.259123,01
2,Adamsville,26.215,0.209,33.602315,-86.971527,01
3,Addison,3.766,0.032,34.202681,-87.178004,01
4,Akron,0.686,0.005,32.879066,-87.740899,01
5,Alabaster,25.279,0.314,33.244399,-86.847372,01
...,...,...,...,...,...,...
31605,Wamsutter,1.627,0.000,41.667977,-107.979142,56
31609,Wheatland,4.098,0.000,42.051662,-104.959465,56
31613,Worland,4.588,0.054,44.020402,-107.961653,56
31614,Wright,3.039,0.000,43.749153,-105.494834,56


In [110]:
city_basic = city_basic.merge(city_geographic, on=['city_name', 'state_id'])
city_basic

,city_id,city_name,county_id,state_id,land_sqmi,water_sqmi,lat,lon
0,01-00124,Abbeville,01067,01,15.543,0.042,31.564724,-85.259123
1,01-00460,Adamsville,01073,01,26.215,0.209,33.602315,-86.971527
2,01-00484,Addison,01133,01,3.766,0.032,34.202681,-87.178004
3,01-00676,Akron,01065,01,0.686,0.005,32.879066,-87.740899
4,01-00820,Alabaster,01117,01,25.279,0.314,33.244399,-86.847372
...,...,...,...,...,...,...,...,...
20712,56-81300,Wamsutter,56037,56,1.627,0.000,41.667977,-107.979142
20713,56-83040,Wheatland,56031,56,4.098,0.000,42.051662,-104.959465
20714,56-84925,Worland,56043,56,4.588,0.054,44.020402,-107.961653
20715,56-85015,Wright,56005,56,3.039,0.000,43.749153,-105.494834
